# Assignment 3a: Basic Gradio RAG Frontend
## Day 6 Session 2 - Building Simple RAG Applications

In this assignment, you'll build a simple Gradio frontend for your RAG system with just the essential features:
- Button to initialize the vector database
- Search query input and button
- Display of AI responses

**Learning Objectives:**
- Create basic Gradio interfaces
- Connect RAG backend to frontend
- Handle user interactions and database initialization
- Build functional AI-powered web applications

**Prerequisites:**
- Completed Assignment 1 (Vector Database Basics)
- Completed Assignment 2 (Advanced RAG)
- Understanding of LlamaIndex fundamentals


## 📚 Part 1: Setup and Imports

Import all necessary libraries for building your Gradio RAG application.


In [2]:
# LB 20251208
!pwd
!git clone https://github.com/lakshmananb/ai-accelerator-C3/

/content
Cloning into 'ai-accelerator-C3'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 161 (delta 34), reused 73 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (161/161), 26.08 MiB | 30.76 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [3]:
!ls
!ls /content/ai-accelerator-C3/Day_7/session_2/
!pip install -r /content/ai-accelerator-C3/Day_7/session_2/requirements.txt

ai-accelerator-C3  sample_data
assignments  llamaindex_rag  prompt_samples    youtube_video
data	     papers	     requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 44.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 19.6 MB/s eta 0:00:00
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for

In [12]:
# LB 20251208:  Below line added for Openrouter API Key configuration (OutSkill provided one fails at Simiarity filtering stage)
# LB 20251208:  API key fixed by generating new personal key

import os
from getpass import getpass

os.environ["OPENROUTER_API_KEY"] = getpass("") #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 (Outskill) / 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 (Personal)
print("✓ OpenrRouter key set successfully")

··········
✓ OpenrRouter key set successfully


In [5]:
# Import required libraries
import gradio as gr
import os
from pathlib import Path

# LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


## 🤖 Part 2: RAG Backend Class

Create a simple RAG backend that can initialize the database and answer queries.


In [20]:
# LB20251210:  Added based on ChatGPT based debug
from llama_index.core.llms import CustomLLM
import requests

class OpenRouterLLM(CustomLLM):
    def __init__(self, api_key, model="gpt-4o", temperature=0.2):
        self.api_key = api_key
        self.model = model
        self.temperature = temperature

    @property
    def metadata(self):
        return {}

    def complete(self, prompt, **kwargs):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "HTTP-Referer": "http://localhost",
            "X-Title": "LlamaIndex-RAG"
        }

        payload = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature
        }

        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=60
        )

        data = response.json()
        return data["choices"][0]["message"]["content"]

print("✅ OpenRouter API setup successfully!")

✅ OpenRouter API setup successfully!


In [30]:
# LB20251210:  Added based on ChatGPT based debug: For OpenRouter API fallback for 401/439/500
import requests
from llama_index.core.llms import CustomLLM
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


class OpenRouterLLMWithFallback(CustomLLM):
    def __init__(self, api_key, model="gpt-4o", temperature=0.2):
        self.api_key = api_key
        self.model = model
        self.temperature = temperature

        # --- Load HuggingFace fallback ---
        self.fallback_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
        self.fallback_model = AutoModelForCausalLM.from_pretrained(
            "microsoft/Phi-3-mini-4k-instruct",
            torch_dtype=torch.float16,
            device_map="auto"
        )

    @property
    def metadata(self):
        return {}

    def complete(self, prompt, **kwargs):
        """
        Try OpenRouter. If it fails → run local HuggingFace LLM.
        """
        # -------------- Try OpenRouter --------------
        try:
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "HTTP-Referer": "http://localhost",
                "X-Title": "LlamaIndex-RAG"
            }

            payload = {
                "model": self.model,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": self.temperature
            }

            response = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers=headers,
                json=payload,
                timeout=30
            )

            if response.status_code == 200:
                data = response.json()
                return data["choices"][0]["message"]["content"]
            else:
                print(f"[⚠️ OpenRouter Error] {response.status_code}: {response.text}")
                raise RuntimeError("Primary LLM failed")

        except Exception as e:
            print(f"[⚠️ Falling back to HuggingFace] {e}")

        # -------------- FALLBACK: HuggingFace local LLM --------------
        inputs = self.fallback_tokenizer(prompt, return_tensors="pt").to(self.fallback_model.device)
        output = self.fallback_model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=self.temperature
        )
        return self.fallback_tokenizer.decode(output[0], skip_special_tokens=True)

print("✅ HuggingFace fallback for OpenRouter API failure for 401/429/500 setup successfully!")


✅ HuggingFace fallback for OpenRouter API failure for 401/429/500 setup successfully!


In [34]:
# LB20251210:  Added based on ChatGPT based debug: For OpenRouter API fallback for 401/439/500 with Stream support
from llama_index.core.llms import CustomLLM, CompletionResponse, CompletionResponseGen
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


class OpenRouterLLMWithFallback(CustomLLM):
    def __init__(self, api_key, model="gpt-4o", temperature=0.2):
        self.api_key = api_key
        self.model = model
        self.temperature = temperature

        # Load HuggingFace fallback
        self.fallback_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
        self.fallback_model = AutoModelForCausalLM.from_pretrained(
            "microsoft/Phi-3-mini-4k-instruct",
            torch_dtype=torch.float16,
            device_map="auto"
        )

    @property
    def metadata(self):
        return {"model_name": f"OpenRouter({self.model}) + HF-Fallback"}

    # ----------------------------------------------------
    # MAIN NON-STREAMING COMPLETION
    # ----------------------------------------------------
    def complete(self, prompt, **kwargs) -> CompletionResponse:
        """
        Try OpenRouter → if it fails, fallback to local HuggingFace.
        """
        try:
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "HTTP-Referer": "http://localhost",
                "X-Title": "LlamaIndex-RAG"
            }

            payload = {
                "model": self.model,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": self.temperature
            }

            response = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers=headers,
                json=payload,
                timeout=25
            )

            if response.status_code == 200:
                data = response.json()
                text = data["choices"][0]["message"]["content"]
                return CompletionResponse(text=text)

            # If request fails → fallback
            raise RuntimeError(f"OpenRouter error: {response.status_code}")

        except Exception as e:
            print(f"[⚠️ OpenRouter failed → fallback triggered] {e}")

        # --------- FALLBACK LOCAL LLM ---------
        inputs = self.fallback_tokenizer(prompt, return_tensors="pt").to(self.fallback_model.device)
        outputs = self.fallback_model.generate(
            **inputs,
            max_new_tokens=250,
            temperature=self.temperature
        )
        text = self.fallback_tokenizer.decode(outputs[0], skip_special_tokens=True)
        return CompletionResponse(text=text)

    # ----------------------------------------------------
    # STREAMING VERSION (required by CustomLLM)
    # ----------------------------------------------------
    def stream_complete(self, prompt, **kwargs) -> CompletionResponseGen:
        """
        Minimal streaming implementation: just yields the full response once.
        """
        full_response = self.complete(prompt).text
        yield full_response
print("✅ HuggingFace fallback for OpenRouter API failure for 401/429/500 setup successfully with minimal STREAM support!")


✅ HuggingFace fallback for OpenRouter API failure for 401/429/500 setup successfully with minimal STREAM support!


In [37]:
class SimpleRAGBackend:
    """Simple RAG backend for Gradio frontend."""

    def __init__(self):
        self.index = None
        self.setup_settings()

    def setup_settings(self):
        """Configure LlamaIndex settings."""
        # Set up the LLM using OpenRouter
        api_key = os.getenv("OPENROUTER_API_KEY")
        if api_key:
            #Settings.llm = OpenRouter(
            #LB20251210:  Fall back added based on debug with help of ChatGPT
            Settings.llm = OpenRouterLLMWithFallback(
                api_key=api_key,
                model="gpt-4o",
                temperature=0.1
            )

        # Set up the embedding model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-small-en-v1.5",
            trust_remote_code=True
        )

        # Set chunking parameters
        Settings.chunk_size = 512
        Settings.chunk_overlap = 50

    def initialize_database(self, data_folder="/content/ai-accelerator-C3/Day_7/session_2/data"):
        """Initialize the vector database with documents."""
        # Check if data folder exists
        if not Path(data_folder).exists():
            return f"❌ Data folder '{data_folder}' not found!"

        try:
            # Create vector store
            vector_store = LanceDBVectorStore(
                uri="/content/ai-accelerator-C3/Day_7/session_2/basic_rag_vectordb",
                table_name="documents"
            )

            # Load documents
            reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
            documents = reader.load_data()

            # Create storage context and index
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            self.index = VectorStoreIndex.from_documents(
                documents,
                storage_context=storage_context,
                show_progress=True
            )

            return f"✅ Database initialized successfully with {len(documents)} documents!"

        except Exception as e:
            return f"❌ Error initializing database: {str(e)}"

    def query(self, question):
        """Query the RAG system and return response."""
        # Check if index exists
        if self.index is None:
            return "❌ Please initialize the database first!"

        # Check if question is empty
        if not question or not question.strip():
            return "⚠️ Please enter a question first!"

        try:
            # Create query engine and get response
            query_engine = self.index.as_query_engine()
            response = query_engine.query(question)
            return str(response)

        except Exception as e:
            return f"❌ Error processing query: {str(e)}"

# Initialize the backend
rag_backend = SimpleRAGBackend()
print("🚀 RAG Backend initialized and ready!")


ValueError: "OpenRouterLLMWithFallback" object has no field "api_key"

## 🎨 Part 3: Gradio Interface

Create a simple Gradio interface with:
1. Button to initialize the database
2. Text input for queries
3. Button to submit queries
4. Text output for responses
5. Text output for status messages


In [27]:
def create_basic_rag_interface():
    """Create basic RAG interface with essential features."""

    def initialize_db():
        """Handle database initialization."""
        return rag_backend.initialize_database()

    def handle_query(question):
        """Handle user queries."""
        return rag_backend.query(question)

    # TODO: Create Gradio interface using gr.Blocks()
    # Hint: Look at the structure below and fill in the missing components

    with gr.Blocks(title="Basic RAG Assistant") as interface:
        # TODO: Add title and description
        # Hint: Use gr.Markdown() for formatted text
        # LB20251210
        gr.Markdown(
            """
            # 📘 Basic RAG Assistant
            This interface lets you **initialize the RAG database** and **run queries** against it.
            """
        )

        # TODO: Add initialization section
        # Hint: You need to use gr.Button to initialize the database
        # init_btn = ?
        # LB20251210
        gr.Markdown("### 📦 Initialize Database")
        init_btn = gr.Button("Initialize Database")

        # TODO: Add status output
        # Hint: You need to use gr.Textbox to display the status
        # LB20251210
        status_output = gr.Textbox(
            label="Status",
            placeholder="Click 'Initialize Database' to begin..."
        )

        # The connection between the button and the status output has already been implemented
        # at the end of this function

        #  status_output = ?
        # LB20251210
        status_output = gr.Textbox(
            label="Status",
            placeholder="Click 'Initialize Database' to begin..."
        )

        # TODO: Add query section
        # Hint: You need a text input, submit button, and response output
        # LB20251210
        gr.Markdown("### 💬 Ask a Question")

        # Use gr.Textbox to create a text input
        # query_input = ?
        # LB20251210
        query_input = gr.Textbox(
            label="Your Question",
            placeholder="Type your question here..."
        )

        # Use gr.Button to create a submit button
        # submit_btn = ?
        # LB20251210
        submit_btn = gr.Button("Submit Query")

        # Use gr.Textbox to create a response output
        # response_output = ?
        # LB20251210
        response_output = gr.Textbox(
            label="Response",
            placeholder="RAG response will appear here..."
        )

        # Connect buttons to functions
        # Uncomment when above is implemented
        # init_btn.click(initialize_db, outputs=[status_output])
        # submit_btn.click(handle_query, inputs=[query_input], outputs=[response_output])
        # LB20251210
        init_btn.click(initialize_db, outputs=[status_output])
        submit_btn.click(handle_query, inputs=[query_input], outputs=[response_output])

    return interface

# Create the interface
basic_interface = create_basic_rag_interface()
print("✅ Basic RAG interface created successfully!")


✅ Basic RAG interface created successfully!


## 🚀 Part 4: Launch Your Application

Launch your Gradio application and test it!


In [28]:
print("🎉 Launching your Basic RAG Assistant...")
print("🔗 Your application will open in a new browser tab!")
print("")
print("📋 Testing Instructions:")
print("1. Click 'Initialize Database' button first")
print("2. Wait for success message")
print("3. Enter a question in the query box")
print("4. Click 'Ask Question' to get AI response")
print("")
print("💡 Example questions to try:")
print("- What are the main topics in the documents?")
print("- Summarize the key findings")
print("- Explain the methodology used")
print("")
print("🚀 Launch your app:")

# Your launch code here:
# Uncomment when implemented
# basic_interface.launch()
# LB20251210
basic_interface.launch()

🎉 Launching your Basic RAG Assistant...
🔗 Your application will open in a new browser tab!

📋 Testing Instructions:
1. Click 'Initialize Database' button first
2. Wait for success message
3. Enter a question in the query box
4. Click 'Ask Question' to get AI response

💡 Example questions to try:
- What are the main topics in the documents?
- Summarize the key findings
- Explain the methodology used

🚀 Launch your app:
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0c70fca338df84a5aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
# LB20251210:  Debug
os.getenv("OPENROUTER_API_KEY")

'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'

**Observations:**
1. With OutSkill provider OpenRouter key, following error encountered at the lsat stage.
❌ Error processing query: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}
2.  With personal OpenRouter key, works fine.

## ✅ Assignment Completion Checklist

Before submitting, ensure you have:

- [x] RAG backend is provided and working
- [x] Created Gradio interface with required components:
  - [x] Title and description using gr.Markdown()
  - [] Initialize database button using gr.Button()
  - [ ] Status output using gr.Textbox()
  - [ ] Query input field using gr.Textbox()
  - [ ] Submit query button using gr.Button()
  - [ ] Response output area using gr.Textbox()
- [ ] Connected buttons to backend functions using .click()
- [ ] Successfully launched the application
- [ ] Tested the full workflow (initialize → query → response)

## 🎊 Congratulations!

You've successfully built your first Gradio RAG application! You now have:

- A functional web interface for your RAG system
- Understanding of Gradio basics and component connections
- A foundation for building more complex AI applications

**Next Steps**: Complete Assignment 3b to add advanced configuration options to your RAG interface!
